## Initialization

In [ ]:
import sys
#sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Modules')
sys.path.append('I:/내 드라이브/backup/document/USC/Research/MCPC/Code/Codes')

#Import packages
#imports
import time
import pandas as pd
from collections import Counter
import sknetwork


from sknetwork.ranking import PageRank
from sknetwork.ranking import Betweenness
from sknetwork.ranking import Closeness

#import umap as umap

from numba.typed import List
import warnings
from numba import njit
import pynndescent
import numpy as np
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import operator
from sklearn.utils.extmath import randomized_svd
from random import randint
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import scipy
#from umap.umap_ import *
import math
from random import randint
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import fashion_mnist
import scanpy
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score, roc_auc_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

#import community as community_louvain
from sklearn.cluster import KMeans
from sklearn.metrics import fowlkes_mallows_score

import scipy

import igraph 
import networkx as nx


import metric as met 
import embedding as embed   
import data_utils_ch as data_util

%load_ext autoreload
%autoreload 2
import importlib

import metric as met 
import embedding as embed   
import data_utils_ch as data_util
from subgraph import *


met=importlib.reload(met)
embed=importlib.reload(embed)
data_util=importlib.reload(data_util)

## Clean Version

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
data_names = ['Zhengmix8eq']
#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data in data_names: 
    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'
    data_for_plot = defaultdict(lambda: defaultdict(dict)) #nested dictionary to store data for plots

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)
    print('Number of nodes:',G.number_of_nodes())

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:
                ''' 
                Louvain on G (baseline)
                '''
                label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
                Num_Cluster_Louv_on_G = len(set(label_))

                '''
                Leiden on G (Baseline)
                '''
                label_, NMI_baseline_leiden, Purity_baseline_leiden = do_Leiden(G, G, res, 0, label)
                Num_Cluster_Leiden_on_G = len(set(label_))

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Louvain on the subgraph H 
                    '''
                    H_label, _, _ = do_Louvain(H, G, res, 0, label)
                    
                    Num_Cluster_Louv_on_H = len({x for x in set(H_label) if x != -1})
                    
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num = len([x for x in H_label if x != -1])
                    '''
                    Plots
                    '''
                    #at index [res][FR_type] store the data for the plot
                    data_for_plot[idx][FR_type] = {'NMI_after_vote':NMI_List[-1], 'Purity_after_vote':Purity_List[-1], 'Louvain_NMI':NMI_, 
                                                    'Louvain_Purity':Purity_, 'Leiden_NMI':NMI_baseline_leiden, 'Leiden_Purity':Purity_baseline_leiden, 
                                                    'Num_Cluster_Louv_on_G':Num_Cluster_Louv_on_G, 'Num_Cluster_Leiden_on_G':Num_Cluster_Leiden_on_G, 
                                                    'Num_Cluster_Louv_on_H':Num_Cluster_Louv_on_H, 'percent_nodes':(selected_node_num*100 / G.number_of_nodes()),
                                                    }
                    
    plot_to_pdf(resolution_list, data_for_plot, k, True_num_of_clusters, pdf_name, data)
    
        
            

                                
                    

                    
                    



## Leiden + Strong Majority Vote + Random Walk on unselected nodes

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05,0.25, 0.5, 1, 1.5, 5]

for data in data_names: 
    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/(Vote+RandWalk)'+data+'.pdf'
    data_for_plot = defaultdict(lambda: defaultdict(dict)) #nested dictionary to store data for plots

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)
    print('Number of nodes:',G.number_of_nodes())

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:
                ''' 
                Louvain on G (baseline)
                '''
                label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
                Num_Cluster_Louv_on_G = len(set(label_))

                '''
                Leiden on G (Baseline)
                '''
                label_, NMI_baseline_leiden, Purity_baseline_leiden = do_Leiden(G, G, res, 0, label)
                Num_Cluster_Leiden_on_G = len(set(label_))

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Louv_on_H = len({x for x in set(H_label) if x != -1})
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    # list of unselected_nodes
                    unselected_nodes = [node_idx for node_idx, node_label in enumerate(H_label) if node_label == -1]
                    selected_node_num = G.number_of_nodes() - len(unselected_nodes)
                    '''
                    Random walks from unselected nodes
                    '''
                    H_label, NMI_rand_walk, Purity_rand_walk = do_random_walks(G, unselected_nodes, label, H_label, node2FR) 
                    selected_node_num2 = len([x for x in H_label if x != -1])
                    #print('selected_before_randwalk:',selected_node_num, 'selected_after_randwalk:',selected_node_num2)
                    '''
                    Plots
                    '''
                    #at index [res][FR_type] store the data for the plot
                    data_for_plot[idx][FR_type] = {'NMI_after_vote':NMI_List[-1], 'Purity_after_vote':Purity_List[-1], 'Louvain_NMI':NMI_, 
                                                    'Louvain_Purity':Purity_, 'Leiden_NMI':NMI_baseline_leiden, 'Leiden_Purity':Purity_baseline_leiden, 
                                                    'Num_Cluster_Louv_on_G':Num_Cluster_Louv_on_G, 'Num_Cluster_Leiden_on_G':Num_Cluster_Leiden_on_G, 
                                                    'Num_Cluster_Louv_on_H':Num_Cluster_Louv_on_H, 'percent_nodes':(selected_node_num*100 / G.number_of_nodes()),
                                                    'extra_nmi':NMI_rand_walk, 'extra_purity':Purity_rand_walk,
                                                    'extra_name':'+RandWalk', 'extra_percent_nodes':(selected_node_num2*100 / G.number_of_nodes())
                                                    }
                    
    plot_to_pdf(resolution_list, data_for_plot, k, True_num_of_clusters, pdf_name, data)
    
        
            

                                
                    

                    
                    



## Test excel

In [1]:
from collections import defaultdict
from subgraph import *
data = defaultdict(list)
data = {'method_name':'testing', 'data_name':'testt'}
write_to_excel(data, 1, 3, 'results.xlsx', 'results')